In [1]:
import random
from pymilvus import connections, CollectionSchema, FieldSchema, DataType, Collection, utility

connections.connect(host='127.0.0.1', port='19530')

In [2]:
book_id = FieldSchema(
    name="book_id",
    dtype=DataType.INT64,
    is_primary=True,
)
book_name = FieldSchema(
    name="book_name",
    dtype=DataType.VARCHAR,
    max_length=200,
)
word_count = FieldSchema(
    name="word_count",
    dtype=DataType.INT64,
)
book_intro = FieldSchema(
    name="book_intro",
    dtype=DataType.FLOAT_VECTOR,
    dim=2
)
schema = CollectionSchema(
    fields=[book_id, book_name, word_count, book_intro],
    description="Test book search",
)

collection = Collection(
    name="book_table",
    schema=schema,
    using='default'
)

collection.create_index(
    field_name="book_intro",
    index_params={
        "metric_type": "L2",
        "index_type": "IVF_FLAT",
        "params": {"nlist": 1024}
    }
)

collection.create_index(
    field_name="book_name",
    index_name="scalar_index",
)

collection.load()

collection.insert([[i for i in range(2000)],
                   [str(i) for i in range(2000)],
                   [i for i in range(10000, 12000)],
                   [[random.random() for _ in range(2)] for _ in range(2000)]])

(insert count: 2000, delete count: 0, upsert count: 0, timestamp: 442749867213193217, success count: 2000, err count: 0)

In [3]:
# Conduct a hybrid vector search
search_param = {
    "data": [[0.1, 0.2]],
    "anns_field": "book_intro",
    "param": {
        "metric_type": "L2",
        "params": {"nprobe": 10,
                   "offset": 5}},
    "limit": 10,
    # By specifying the boolean expression, you can filter the scalar field of the entities during the vector search. The following example limits the scale of search to the vectors within a specified word_count value range.
    "expr": "word_count <= 11000",
    "output_fields": ['book_id', 'book_name', 'word_count']
}
results = collection.search(**search_param)

In [4]:
for i in range(10):
    hit = results[0][i]
    print(hit)

id: 201, distance: 2.0475592464208603e-05, entity: {'book_id': 201, 'book_name': '201', 'word_count': 10201}
id: 598, distance: 2.5180794182233512e-05, entity: {'book_id': 598, 'book_name': '598', 'word_count': 10598}
id: 597, distance: 2.699909600778483e-05, entity: {'book_id': 597, 'book_name': '597', 'word_count': 10597}
id: 833, distance: 2.749182931438554e-05, entity: {'book_id': 833, 'book_name': '833', 'word_count': 10833}
id: 828, distance: 4.841619738726877e-05, entity: {'book_id': 828, 'book_name': '828', 'word_count': 10828}
id: 604, distance: 5.6165743444580585e-05, entity: {'book_id': 604, 'book_name': '604', 'word_count': 10604}
id: 51, distance: 5.969862831989303e-05, entity: {'book_id': 51, 'book_name': '51', 'word_count': 10051}
id: 433, distance: 6.296827632468194e-05, entity: {'book_id': 433, 'book_name': '433', 'word_count': 10433}
id: 789, distance: 6.53224706184119e-05, entity: {'book_id': 789, 'book_name': '789', 'word_count': 10789}
id: 142, distance: 8.12969519

In [5]:
collection.release()